In [1]:
import sys
#import os
sys.path.append('../')

from utils.functions import *

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

In [2]:
path = '../Data/Images'

In [167]:
def get_data(path):
    
    folders = os.listdir(path)
    
    data = []
    
    for folder in folders:
    
        # Label as healthy or diseased
        if folder.endswith('healthy'):
            label = 1.
        else:
            label = 0.

        # Iterate through the folders
        folder_images = os.listdir(path + '/' + folder)
        folder_path = path + '/' + folder + '/'

        # Extract images from each folder
        for image in folder_images:
            image_path = os.path.join(folder_path, image)
            image_path = Image.open(image_path)

            # Convert image to pixel values
            pixel_values = np.array(image_path) / 255

            # Add image and label
            #data.append((pixel_values, label))
            img = torch.tensor(pixel_values).to(dtype = torch.float32).clone().detach().requires_grad_(True)
            img = img.permute(2,1,0)
            lab = torch.tensor(np.array(label), dtype=torch.long)
            
            data.append((img, lab))
        
    return data

In [168]:
def train_val_test_split(data):
    
    # Shorten data to 10,000 images
    #data = data[:10000]
    
    # Shuffle the data
    random.shuffle(data)
    
    # Define the proportions for train, validation, and test sets
    train_ratio = 0.6
    val_ratio = 0.2
    test_ratio = 0.2
    
    # Calculate the number of samples for each set
    num_samples = len(data)
    num_train = int(train_ratio * num_samples)
    num_val = int(val_ratio * num_samples)
    num_test = num_samples - num_train - num_val
    
    # Split the images
    train = images[:num_train]
    val = images[num_train:num_train + num_val]
    test = images[num_train + num_val:]
    
    return train, val, test

In [169]:
data = get_data(path)

In [170]:
# Shuffle the data
random.shuffle(data)

In [171]:
data = data[:3000]

In [172]:
data[0]

(tensor([[[0.3647, 0.3490, 0.3647,  ..., 0.3804, 0.4196, 0.3373],
          [0.3725, 0.3490, 0.3490,  ..., 0.3216, 0.3647, 0.3020],
          [0.3765, 0.3490, 0.3373,  ..., 0.4549, 0.4353, 0.2745],
          ...,
          [0.7922, 0.7961, 0.8000,  ..., 0.6745, 0.6275, 0.5529],
          [0.7922, 0.7961, 0.8000,  ..., 0.6706, 0.6196, 0.5412],
          [0.7922, 0.7961, 0.8000,  ..., 0.6784, 0.6314, 0.5490]],
 
         [[0.4706, 0.4549, 0.4706,  ..., 0.3451, 0.3804, 0.2980],
          [0.4784, 0.4549, 0.4549,  ..., 0.2902, 0.3333, 0.2706],
          [0.4824, 0.4549, 0.4431,  ..., 0.4392, 0.4275, 0.2667],
          ...,
          [0.7882, 0.7922, 0.7961,  ..., 0.6667, 0.6196, 0.5451],
          [0.7882, 0.7922, 0.7961,  ..., 0.6627, 0.6118, 0.5333],
          [0.7882, 0.7922, 0.7961,  ..., 0.6706, 0.6235, 0.5412]],
 
         [[0.4039, 0.3882, 0.4039,  ..., 0.1922, 0.2392, 0.1569],
          [0.4118, 0.3882, 0.3882,  ..., 0.1373, 0.1882, 0.1255],
          [0.4157, 0.3882, 0.3765,  ...,

In [173]:
# Define the proportions for train, validation, and test sets
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

num_samples = len(data)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

# Split the images
train = data[:num_train]
val = data[num_train:num_train + num_val]
test = data[num_train + num_val:]

In [174]:
train[0][0].shape

torch.Size([3, 256, 256])

In [175]:
len(train)

1800

### CNN model

In [176]:
# Size of minibatch for minibatch SGD
batch_size = 32

In [177]:
# Convert to dataloaders
train_loader = DataLoader(train, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(val)
test_loader = DataLoader(test)

In [178]:
next(iter(train_loader))

[tensor([[[[0.3647, 0.3490, 0.3647,  ..., 0.3804, 0.4196, 0.3373],
           [0.3725, 0.3490, 0.3490,  ..., 0.3216, 0.3647, 0.3020],
           [0.3765, 0.3490, 0.3373,  ..., 0.4549, 0.4353, 0.2745],
           ...,
           [0.7922, 0.7961, 0.8000,  ..., 0.6745, 0.6275, 0.5529],
           [0.7922, 0.7961, 0.8000,  ..., 0.6706, 0.6196, 0.5412],
           [0.7922, 0.7961, 0.8000,  ..., 0.6784, 0.6314, 0.5490]],
 
          [[0.4706, 0.4549, 0.4706,  ..., 0.3451, 0.3804, 0.2980],
           [0.4784, 0.4549, 0.4549,  ..., 0.2902, 0.3333, 0.2706],
           [0.4824, 0.4549, 0.4431,  ..., 0.4392, 0.4275, 0.2667],
           ...,
           [0.7882, 0.7922, 0.7961,  ..., 0.6667, 0.6196, 0.5451],
           [0.7882, 0.7922, 0.7961,  ..., 0.6627, 0.6118, 0.5333],
           [0.7882, 0.7922, 0.7961,  ..., 0.6706, 0.6235, 0.5412]],
 
          [[0.4039, 0.3882, 0.4039,  ..., 0.1922, 0.2392, 0.1569],
           [0.4118, 0.3882, 0.3882,  ..., 0.1373, 0.1882, 0.1255],
           [0.4157, 0.38

In [179]:
def accuracy(model, test_loader):

    correct = 0
    total = 0

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            print(outputs.item())
            predicted = torch.round(outputs) 
            
            
            # Convert predictions and labels to numpy arrays for comparison
            predicted = predicted.numpy().flatten()
            labels = labels.numpy().flatten()
            
            # Count the number of correct predictions
            correct += (predicted == labels).sum().item()
            total += labels.size
            
    # Calculate the accuracy
    accuracy = correct / total

    return accuracy

In [180]:
# model = nn.Sequential(
#     nn.Conv2d(3, 16, kernel_size=4, padding=1, stride=2),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     nn.Conv2d(16, 32, kernel_size=4, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     nn.Conv2d(32, 64, kernel_size=4, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     nn.Conv2d(64, 128, kernel_size=4, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     nn.Conv2d(128, 256, kernel_size=4, padding=1),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     nn.Flatten(),
#     nn.Linear(2304, 512),
#     nn.ReLU(),
#     nn.Linear(512, 128),
#     nn.ReLU(),
#     nn.Linear(128, 1),
#     nn.Sigmoid()
# )

In [181]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=4),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.8),
    nn.Conv2d(32, 64, kernel_size=4),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.8),
    nn.Conv2d(64, 128, kernel_size=4),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.8),
    nn.Conv2d(128, 256, kernel_size=4),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.8),
    nn.Conv2d(256, 128, kernel_size=4),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Dropout(0.8),
    nn.Flatten(),
    nn.Linear(3200, 512),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

In [182]:
# Instantiate the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [189]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        outputs = model(images)
        #print(outputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

    #acc = accuracy(model, val_loader)

    #print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 1:   0%|                                                                                  | 0/56 [00:00<?, ?it/s]

tensor([[0.0059],
        [0.7604],
        [0.1593],
        [0.9167],
        [0.7541],
        [0.0382],
        [0.7919],
        [0.6843],
        [0.4262],
        [0.0767],
        [0.6242],
        [0.0139],
        [0.6932],
        [0.0181],
        [0.1506],
        [0.1813],
        [0.3703],
        [0.9181],
        [0.6904],
        [0.7752],
        [0.1703],
        [0.3557],
        [0.0726],
        [0.5904],
        [0.4181],
        [0.0972],
        [0.5561],
        [0.6534],
        [0.7130],
        [0.3783],
        [0.7306],
        [0.1656]], grad_fn=<SigmoidBackward0>)


Epoch 1:   2%|█▎                                                                        | 1/56 [00:05<05:16,  5.75s/it]

tensor([[0.3950],
        [0.4916],
        [0.6058],
        [0.0144],
        [0.3534],
        [0.2780],
        [0.2758],
        [0.3627],
        [0.3800],
        [0.4126],
        [0.0175],
        [0.6181],
        [0.3127],
        [0.6643],
        [0.5987],
        [0.3339],
        [0.3387],
        [0.6568],
        [0.1305],
        [0.5335],
        [0.2755],
        [0.2763],
        [0.2642],
        [0.5634],
        [0.1920],
        [0.4431],
        [0.6573],
        [0.7767],
        [0.2383],
        [0.3313],
        [0.1879],
        [0.5662]], grad_fn=<SigmoidBackward0>)


Epoch 1:   4%|██▋                                                                       | 2/56 [00:11<05:06,  5.68s/it]

...

tensor([[0.4652],
        [0.4914],
        [0.3626],
        [0.5150],
        [0.5426],
        [0.4449],
        [0.4250],
        [0.4122],
        [0.5626],
        [0.4264],
        [0.4710],
        [0.4856],
        [0.5394],
        [0.5037],
        [0.5166],
        [0.4894],
        [0.5140],
        [0.4633],
        [0.4560],
        [0.5312],
        [0.4341],
        [0.5200],
        [0.5129],
        [0.4895],
        [0.3662],
        [0.4840],
        [0.4715],
        [0.4192],
        [0.4578],
        [0.4534],
        [0.4420],
        [0.5005]], grad_fn=<SigmoidBackward0>)


Epoch 1:  12%|█████████▎                                                                | 7/56 [00:40<04:40,  5.73s/it]

tensor([[0.5600],
        [0.3577],
        [0.4941],
        [0.5444],
        [0.4944],
        [0.5308],
        [0.4407],
        [0.5071],
        [0.4445],
        [0.3974],
        [0.5284],
        [0.4238],
        [0.4418],
        [0.4771],
        [0.5475],
        [0.4665],
        [0.4502],
        [0.5225],
        [0.4840],
        [0.4514],
        [0.5253],
        [0.4833],
        [0.4539],
        [0.5021],
        [0.4574],
        [0.4424],
        [0.5557],
        [0.4548],
        [0.5230],
        [0.5967],
        [0.4763],
        [0.4072]], grad_fn=<SigmoidBackward0>)


Epoch 1:  12%|█████████▎                                                                | 7/56 [00:46<05:22,  6.58s/it]


KeyboardInterrupt: 

In [ ]:
accuracy(model, val_loader)